In [1]:
# install packages
!pip install pytesseract
!pip install pdf2image

You should consider upgrading via the 'C:\Users\lmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\lmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
# import packages
import pytesseract
from pdf2image import convert_from_path
import os
import re
from datetime import datetime
import pandas as pd

In [3]:
# Function to extract text from a PDF file
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  

def extract_text_from_pdf(pdf_path):
    # Convert the PDF to a list of PIL Image objects
    images = convert_from_path(pdf_path)
    text_list = []
    # Loop through each page of the PDF and extract text using Tesseract
    for image in images:
        text = pytesseract.image_to_string(image)
        text_list.append(text)
    # Combine text from all pages into a single string
    text = '\n'.join(text_list)
    return text

In [243]:
# Define the regex pattern to match the date
date_pattern = r'\b\d{1,2}\s+(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}\b'
#council_pattern = r'(?i)(\b[A-Za-z]+\s(?:(\n)Council|(\n)Committee|(\n)COMMITTEE)\b)'
council_pattern = r"(?<=session).*?(?=Agenda)"
council_pattern2 = r'([A-Z]+\s)+COUNCIL'
session_pattern = r'(?P<session>[\w-]+\ssession)'
agenda_pattern = r'Agenda item (\d+)(?: \((\w+)\))?(?:\n\n)?'
draft_pattern = r'draft(?:\n\n)?\s?resolution'
#agenda_detail_pattern = r'Agenda item\s+(\d+)\s+\(.+\).*?(draft(?:\n\n)?\s?resolution)' #(?<=Agenda item \d+ \(.\)\n).*?(?=draft(?:\n)?resolution|\n\n)'
footnote_pattern = r'^(\*|\d(?![./])|\d{1}/)\s.*'
title_pattern = r"1\.\s+\w+(\s+\w+)*"
text_head_pattern = r'(The General Assembly,|The Commission on Human Rights,|1\.)'
number_title_pattern = r'^\s*(\d{4})?/?\W*(.*)'

In [167]:
# extract year (A) and date (J)
def extract_date(text, pattern):
    match = re.search(pattern, text)
    if match:
        date = match.group(0)
        date_obj = datetime.strptime(date, '%d %B %Y')
        # change mm/dd/yyyy format
        new_date = date_obj.strftime('%m/%d/%Y') #column J
        year = new_date[-4:] #column A
    else:
        year, date = 'N/A'
        print("No date found")
    return year, date

In [251]:
def remove_empty(text):
    text = text.replace('\n', ' ')
    if text == '' or text == ' ':
        return 'N/A'
    else:
        return text
    
# extract council/committe name (B)
def extract_council(text, pattern1, pattern2):
    match1 = re.search(pattern1, text, re.DOTALL)
    match2 = re.search(pattern2, text, re.DOTALL)
    council = 'N/A'
    # handle the council after session
    if match1:
        council = match1.group(0).strip()
    # handle the council before session
    if match2:
        council = match2.group(0).strip()
    return remove_empty(council)

In [170]:
# Function to classify text into column A to N
def classify_text_to_column(text):
    result = []

    # get year(A) and date(J)
    year, date = extract_date(text, date_pattern)
    result.append(year)


    # get council/committe name (B)
    council = extract_council(text, council_pattern)
    result.append(council)


    # extract session (C)
    match = re.search(session_pattern, text)
    if match:
        session = match.group("session")
    else:
        session = 'N/A'
        print("Session information not found.")
    result.append(session)


    # extract Agenda item (D), Agenda detail (E), and countries (F)
    agenda_match = re.search(agenda_pattern, text)
    if match:
        agenda_item_number = agenda_match.group(1)
        agenda_item_letter = agenda_match.group(2)
        if agenda_item_letter is not None:
            agenda_item = f"{agenda_item_number} ({agenda_item_letter})"
        else:
            agenda_item = agenda_item_number
        result.append(agenda_item)
        
        start_index = agenda_match.end()
        draft_match = re.search(draft_pattern, text[start_index:], re.DOTALL)
        if draft_match:
            end_index = start_index + draft_match.start()
            content = text[start_index:end_index]
            #print(content)
            parts = content.split("\n\n")
            #print(parts)
            parts = [s.replace('\n', '') for s in parts]
            result.append(parts[0])
            result.append(parts[1][:-2])
    else:
        agenda_item = 'N/A'
        print("No match found")
        result.append(agenda_item)


    # delete extracted text
    split_text = re.split(draft_pattern, text, flags=re.IGNORECASE)
    remain_text = split_text[1]
    #print(remain_text)

    # body title number (G) and body title （H）
    title_body_text = re.split(text_head_pattern, remain_text)
    title_text = title_body_text[0]
    #match = re.search(r'[A-Z]', title_text)
    title_text = title_text.replace('\n', ' ')
    # print(title_text)
    match = re.match(number_title_pattern, title_text)
    if match:
        title_number = match.group(1)
        if title_number == None:
            title_number = 'N/A'
        title = match.group(2)
    else:
        title_number, title = 'N/A'

    result.append(title_number)
    result.append(title)


    # Body text (I)
    # extract the text content from the second non-empty paragraph
    body_text = ''
    footnote = ''
    footnote_idx = 1
    for paragraph in paragraphs[idx:]:
        #for line in paragraph.split('\n'):
            # exclude lines containing specific patterns
        paragraph = paragraph.replace("\n", " ")
        match = re.search(footnote_pattern, paragraph)
        if match:
            if match.group(0)[0] == '*':
                footnote += match.group(0)
            elif len(match.group(0)) < 90 and match.group(0)[0] == str(footnote_idx):
                footnote += match.group(0)
                footnote_idx += 1
        else:
            #if not re.search(r'^(\d{2}-\d{5}|[A-Za-z]/\d{2}/[A-Za-z]\.\d|[A-Za-z ]+Please recycle|Page \d{1})', paragraph):
            if len(paragraph) > 50 or paragraph == "The General Assembly,":
                body_text += paragraph.strip() + ' '
            else:
                if len(paragraph) == 0:
                    continue
                if paragraph[0] == str(footnote_idx) or paragraph[0] == "*":
                    footnote_idx += 1
                    footnote += paragraph 
    result.append(body_text)

    # date (J)
    result.append(date)
    
    # # file name (K) and country (L) will be obtained outside

    # footnote (M)
    result.append(footnote)

    # scanned (N), cannot know 
    if int(year) > 1990:
        scanned = 'yes'
    else:
        scanned = 'no'
    result.append(scanned)
    
    return result

In [107]:
# Path to the folder containing the PDF files
pdf_folder = './Dataset'

# Path to the folder where the CSV files will be saved
csv_folder = './csv_folder'

# csv file column list
column_list = ['year', 'Council', 'Session', 'Agenda item', 'Agenda detail', 'cosponsored countries', 
               'body title number',	'body title detail', 'body text', 'date', 'footnote', 'scanned', 
               'file', 'filecountry']

# Loop through each folder and PDF file and extract text
for country_folder in os.listdir(pdf_folder):
    print('---------------' + country_folder + '---------------' )
    if not os.path.isdir(os.path.join(pdf_folder, country_folder)):
        continue
    csv_path = os.path.join(csv_folder, f"{country_folder}.csv")
    rows = []
    count = 0
    # Loop through each file 
    for pdf_file in os.listdir(os.path.join(pdf_folder, country_folder)):
        if not pdf_file.endswith('.pdf'):
            continue
        # check if this file is old-version
        year = int(pdf_file.split('_')[0])
        if year < 1994:
            continue

        pdf_path = os.path.join(pdf_folder, country_folder, pdf_file).replace('\\', '/')
        print(pdf_path)
        # try:
        text = extract_text_from_pdf(pdf_path)
        text_columns = classify_text_to_column(text)
        row = text_columns + [pdf_file[:-4], country_folder] #add column K and L
        rows.append(row)
        count += 1
        #print(rows)
        df = pd.DataFrame(rows, columns = column_list)
        df.to_csv(csv_path, index = False) 
        if count == 10:
            break
        # except:
        #     print(f"Error extracting text from {pdf_path}")
        #     continue
    break
    # Write the rows to the CSV file
    df = pd.DataFrame(rows, columns = column_list)
    df.to_csv(csv_path, index = False)  

---------------Afghanistan---------------
./Dataset/Afghanistan/1994_163336.pdf
./Dataset/Afghanistan/1994_163532.pdf
./Dataset/Afghanistan/1994_163619.pdf
./Dataset/Afghanistan/1994_163690.pdf
./Dataset/Afghanistan/1994_163780.pdf
./Dataset/Afghanistan/1994_163787.pdf
./Dataset/Afghanistan/1994_163866.pdf
./Dataset/Afghanistan/1994_163874.pdf
./Dataset/Afghanistan/1994_163890.pdf
./Dataset/Afghanistan/1994_163906.pdf


In [235]:
test3 = extract_text_from_pdf('./Dataset/Afghanistan/2006_584361.pdf')
test3

'General Assembly Distr.\nLIMITED\n\nA/HRC/2/L.46\n3 October 2006\n\nOriginal: ENGLISH\n\nHUMAN RIGHTS COUNCIL\nSecond session\nAgenda item 2\n\nIMPLEMENTATION OF GENERAL ASSEMBLY RESOLUTION 60/251\nOF 15 MARCH 2006 ENTITLED “HUMAN RIGHTS COUNCIL”\n\nFinland (on behalf of the European Union)\nand Afghanistan*: draft decision\n\n2006/... Cooperation with the Office of the United Nations\nHigh Commissioner for Human Rights: Afghanistan\n\nThe Human Rights Council decides to adopt the following text:\n\n“The Human Rights Council welcomes the report of the United Nations\nHigh Commissioner for Human Rights on the situation of human rights in Afghanistan\nand on the achievements of technical assistance in the field of human rights\n(E/CN.4/2006/108), including the assessment of the human rights situation of\nAfghanistan contained therein, and the ongoing cooperation between the Government\nof Afghanistan and the Office of the High Commissioner, as well as with the special\nprocedures of the

In [252]:
extract_council(test3, council_pattern, council_pattern2)

'HUMAN RIGHTS COUNCIL'

### TEST

In [253]:
test_list = ['1994_163532.pdf', '1994_163787.pdf', '1994_163866.pdf', '2000_412394.pdf', '2006_584361.pdf'] 
for test_pdf in test_list:
    #if test_pdf == '2006_584361.pdf':
    test_text = extract_text_from_pdf(f'./Dataset/Afghanistan/{test_pdf}')
        #print(test_text)
    # test year and date
    #print(extract_date(test_text, date_pattern))
    print(extract_council(test_text, council_pattern, council_pattern2))

SPECIAL POLITICAL AND DECOLONIZATION COMMITTEE (FOURTH COMMITTEE)
N/A
THIRD COMMITTEE
N/A
HUMAN RIGHTS COUNCIL


In [177]:
# y, d = extract_date(test, date_pattern)
# print(y)
# print(d)
c = extract_council(test, council_pattern2)
print(c)

N/A


In [164]:
# delete extracted text
split_text = re.split(draft_pattern, test, flags=re.IGNORECASE)
remain_text = split_text[1]

text_head_pattern = r'(The General Assembly,|The Commission on Human Rights,|1\.)'
number_title_pattern = r"^\s*(\d{4})?/?\W*(.*)"

title_body_text = re.split(text_head_pattern, remain_text)
title_text = title_body_text[0]
#match = re.search(r'[A-Z]', title_text)
title_text = title_text.replace('\n', ' ')
#print(title_text)
match = re.match(number_title_pattern, title_text)
if match:
    title_number = match.group(1)
    if title_number == None:
        title_number = 'N/A'
    title = match.group(2)
else:
    title_number = 'N/A'
    title = 'N/A'

print(title_number)
print(title)


# Body title (H)
# split the text into paragraphs
body_title = ''
paragraphs = remain_text.split('\n\n')
body_idx = 1
body_title = ''

# for paragraph in paragraphs[1:]:
#     # check if this paragraph contains the beginning pattern of the body text
#     # lines = remain_text.split('\n')
#     # for line in lines:
#     #     if re.search(text_head_pattern, line):
#     #         break
#     print(paragraph)
#     if re.search(text_head_pattern, paragraph):
#         break
#     else:
#         body_title += paragraph
#         body_idx +=1
# if body_idx ==1:
#     body_title = 'N/A'
# else:
#     body_title = body_title.replace('\n', ' ') 
# print("Body title: ", body_title)


# # Body text (I)
# # extract the text content from the second non-empty paragraph
# body_text = ''
# footnote = ''
# footnote_idx = 1
# for paragraph in paragraphs[body_idx:]:
#     #for line in paragraph.split('\n'):
#         # exclude lines containing specific patterns
#     paragraph = paragraph.replace("\n", " ")
#     match = re.search(footnote_pattern, paragraph)
#     if match:
#         if match.group(0)[0] == '*':
#             footnote += match.group(0)
#         elif len(match.group(0)) < 90 and match.group(0)[0] == str(footnote_idx):
#             footnote += match.group(0)
#             footnote_idx += 1
#     else:
#         #if not re.search(r'^(\d{2}-\d{5}|[A-Za-z]/\d{2}/[A-Za-z]\.\d|[A-Za-z ]+Please recycle|Page \d{1})', paragraph):
#         if len(paragraph) > 50 or paragraph == "The General Assembly,":
#             body_text += paragraph.strip() + ' '
#         else:
#             if len(paragraph) == 0:
#                 continue
#             if (paragraph[0] == str(footnote_idx) and paragraph[1] == '/') or paragraph[0] == "*":
#                 footnote_idx += 1
#                 footnote += paragraph 
#             else:
#                 body_text += paragraph.strip() + ' '
# print(footnote)


N/A
Towards full integration of persons with disabilities in society:  implementation of the Standard Rules on the Equalization of  Opportunities for Persons with Disabilities, and of the long-term  strategy to implement the world programme of action concerning disabled persons to the year 2000 and beyond  


In [205]:
string = 'General Assembly Distr.\nLIMITED\n\nA/HRC/2/L.46\n3 October 2006\n\nOriginal: ENGLISH\n\nHUMAN RIGHTS COUNCIL\nSecond session\nAgenda item 2\n\nIMPLEMENTATION OF GENERAL ASSEMBLY RESOLUTION 60/251\nOF 15 MARCH 2006 ENTITLED “HUMAN RIGHTS COUNCIL”\n\nFinland (on behalf of the European Union)\nand Afghanistan*:'

council_name = re.search(r'^\w+(?:\s+\w+)*\s+COUNCIL', string).group(0)

print(council_name)  # HUMAN RIGHTS COUNCIL

AttributeError: 'NoneType' object has no attribute 'group'